# 3. Constant Error Backprop
## 3.1 Exponentially Decaying Error
### Conventional BPTT
Output  unit $k$'s target at time $t$ is denoted by $d_k(t)$.<br>
Using mean squared error, $k$'s error signal is:<br>
$$
\vartheta_k(t) = f^{\prime} _k(net_k(t))(d_k(t)-y^k(t)),
$$
$$where$$
$$
y^i(t) = f_i(net_i(t))
$$
is the activation of a non-input unit $i$ with differentiable activation function $f_i$,
$$
net_i(t) = \sum_j w_{ij}y^j(t-1)
$$
is unit$i$'s current net input, and $w_{ij}$ is the weight on the connection from unit $j$ to $i$.<br>
Some non-output unit $j$'s backpropagated error signal is
$$
\vartheta_j(t) = f^{\prime}_j(net_j(t)) \sum_i w_{ij} \vartheta_i(t+1)
$$
The corresponding contribution to $w_{ji}$'s total weight update is $\alpha \vartheta_j(t) y^l(t-1)$,<br>
where$\alpha$ is the learning rate, and $l$ stands for an arbitrary unit connected to unit $j$.

In [2]:
import numpy as np

def f_i(x, i):
    return x[i] if x[i] > 0 else 0 # ReLU as an example

def f_prime_i(x, i):
    return 1 if x[i] > 0 else 0

def net_i(t, weights, y, i):
    return sum(weights[i][j] * y[j](t-1) for j in range(len(weights[i])))
        
def yi(t, net, i):
    return f_i(net_i(t, net, y, i), i)
    
def vartheta_k(t, net, d, y, k):
    return f_prime_i(net_i(t, net, y, k), k) * (d[k](t) - y[k](t))
    
def vartheta_j(t, net, weights, theta, j):
    return f_prime_i(net_i(t, net, y, j), j) * sum(weights[i][j] * theta[i](t+1) for i in range(len(weights)))
    

### Outline of Hchreiter's analysis.
Suppose we have a fully connected net whose non-input unit indices range from 1 to $n$.<br>
Let us focus on local error flow from unit $u$ to unit $v$ (later we will see that the analysis immediately extends to global error flow).<br>
The error occuring at an arbitrary unit $u$ at time step $t$ is propagated "Back into time" for $q$ time steps, to an arbitrary unit $v$.<br>
This will scale the error by the following factor:
\begin{equation}
\frac{\partial \vartheta_v(t-q)}{\partial \vartheta_u(t)}=\left\{\begin{array}{cc}
f_v^{\prime}\left(\operatorname{net}_v(t-1)\right) w_{u v} & q=1 \\
f_v^{\prime}\left(\operatorname{net}_v(t-q)\right) \sum_{l=1}^n \frac{\partial \vartheta_l(t-q+1)}{\partial \vartheta_u(t)} w_{l v} & q>1
\end{array} \right.
\end{equation}
With $l_q=v$ and $l_0 = u$, we obtain:
$$
\frac{\partial\vartheta_v(t-q)}{\partial\vartheta_u(t)} = \sum^n_{l1=1}...\sum^n_{lq-1=1}\prod^q_{m=1} f^{\prime}_{l_m}(\operatorname{net}_{l_m}(t-m))w_{l_ml_{m-1}}
$$

(proof by induction).<br>
The sum of the $n^{q-1}$ terms $\Pi^q_{m=1}f^{\prime}_{l_m}(\operatorname{net}_{l_m}(t-m))w_{l_ml_{m-1}}$ determines the total error back flow (note that since the summation terms may have different signs, increasing the number of units $n$ does not necessarily increase error flow).

... I tried to implement all the equation on this chapter,<br>
However, the Chapter 3 is about the problems the previous works have and too much to simply write it down.<br>
I decided to understand them by reading the paper,<br>
and focus on implementing LSTM.

# 4. Long Short-Term Memory.
## Memory Cells and gate units
To construct an architecture that allows for constant error flow through special, self-connected units without disadvantages of the naive approach, we extend the constant error carousel (CEC) embodied by the self-connected, linear unit $j$ from section 3.2 by introducing additional features.<br>
A multiplicative *input gate unit* is  introduced to protect the memory contents stored in $j$ from perturbation by irrelevant inputs.<br>
Likewise, a multiplicative *output gate unit* is introduced which protects other units from perturbation by currently irrelevant memory contents stored in $j$.<br><br>
The resulting, more complex unit is called a *memory cell*.<br>
The $j$-th memory cell is denoted $c_j$.<br>
Each memory cell is built around a central lineear unit with a fixed self-connection (the CEC).<br>
In addition to $net_{c_j}$, $c_j$ gets input from multiplicative unit $out_j$ (the "output gate"), and from another multiplicative unit $in_j$ (the "input gate").<br>
$in_j$'s activation at time $t$ is denoted by $y^{in_j}(t)$, $out_j$'s by $y^{out_j}(t)$.<br>
We have:
\begin{equation}
\begin{aligned}
    y^{out_j}(t) &= f_{out_j}(net_{out_j}(t)) \\
    y^{in_j}(t) &= f_{in_j}(net_{in_j}(t))
\end{aligned}
\end{equation}
$$
\text{where}
$$
$$
net_{out_j}(t) = \sum_u w_{out_{j^u}}y^u(t-1),
$$
$$
\text{and}
$$
$$
net_{in_j}(t) = \sum_u w_{in_{j^u}}y^u(t-1),
$$
$$
\text{we also have}
$$
$$
net_{c_j}(t) = \sum_u w_{c_{j^u}}y^u(t-1),
$$
$$
\text{Jason's note: These all look same to me, which is:}
$$
$$
y_i(t)= \sum_u w_{i^u}y^u(t-1)
$$
The summation indices $u$ may stand for input units, gate units, memory cells, or even conventional hidden units if there are any.<br>
All thes different types of units may convey useful information about the current state of the net.<br>
For instance, an input gate may use inputs from other memory cells to decide whether to store certain information in its memory cell.<br>
There even may be recurrent self-connections like $w_{{c_j}{c_j}}$.<br>
It is up to the user to define the network topology.<br>
At time $t$, $c_j$'s output $y^{c_j}(t)$ is computed as:
$$
y^{c_j}(t) = y^{out_j}(t)h(s_{c_j})(t)),
$$
$$
\text{where the "internal state "}s_{c_j}(t)\text{ is}
$$
\begin{equation}
\begin{aligned}
    s_{c_j}(0) &= 0, \\
    s_{c_j}(t) &= s_{c_j}(t-1) + y^{in_j}(t)g(net_{c_j}(t)) for t > 0,
\end{aligned}
\end{equation}
The differentiable function $g$ squashes $net_{c_j}$; the differentiable function $h$ scales memory cell outputs computed from the internal state $s_{c_j}$

In [5]:
def net_i(t, weights, prev_outputs):
    '''
    t : t
    weights : w
    prev_outputs : y
    '''
    net_input = np.dot(weights[i], prev_outputs)
    return net_input

In [7]:
class MemoryCell:
    def __init__(self, input_units, cell_units):
        self.w_in = np.random.randn(input_units, cell_units)
        self.w_out = np.random.randn(input_units, cell_units)
        self.w_c = np.random.randn(input_units, cell_units)
        self.w_cc = np.random.randn(input_units, cell_units) # self-connection weight

        self.s_c = np.zeros(cell_units)
        self.y_in = np.zeros(cell_units)
        self.y_out = np.zeros(cell_units)
        self.y_c = np.zeros(cell_units)

    def activation(self, x):
        return 1 / (1 + np.exp(-x))

    def gate_activation(self, x):
        return 1 / (1 + np.exp(-x))

    def state_activation(self, x):
        return np.tanh(x)

    def net_input(self, y_previous, w):
        # which is equivalent to the code in a cell above
        return np.dot(y_previous, w)

    def update(self, y_previous):
        net_in = self.net_input(y_previous, self.w_in)
        net_out = self.net_input(y_previous, self.w_out)
        net_c = self.net_input(y_previous, self.w_c) + self.net_input(self.s_c, self.w_cc)

        self.y_in = self.gate_activation(net_in)
        self.y_out = self.gate_activation(net_out)
        
        self.s_c = self.s_c + self.y_in * self.state_activation(net_c)
        self.y_c = self. y_out * self.state_activation(self.s_c)
        return self.y_c        

In [8]:
from tqdm.auto import tqdm
import numpy as np
import re

data = """
나라의 말이 중국과 달라 문자와 서로 통하지 아니하기에 이런 까닭으로 어리석은 백성이 이르고자 할 바가 있어도 마침내 제 뜻을 능히 펴지 못할 사람이 많으니라 내가 이를 위해 가엾이 여겨 새로 스물여덟 글자를 만드노니 사람마다 하여 쉬이 익혀 날로 씀에 편안케 하고자 할 따름이니라
"""

def preprocessing(text):
    data = re.sub('[^가-힣]', ' ', text)
    tokens = data.split()
    vocab = list(set(tokens))
    vocab_size = len(vocab)

    word_to_idx = {word: i for i, word in enumerate(vocab)}
    idx_to_word = {i: word for i, word in enumerate(vocab)}

    return tokens, vocab_size, word_to_idx, idx_to_word

In [ ]:
def sigmoid(input):
    return 1 / (1 + np.exp(-input))

def sigmoid_derivative(input):
    return input * (1 - input)

def tanh(input, derivative=False):
    return np.tanh(input)

def tanh_derivative(input):
    return 1 - input ** 2

def so